# Libraries

In [2]:
import pandas as pd  # Для работы с таблицами данных
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u
from astropy.io import fits
from astropy.table import Table
import numpy as np

In [18]:
hdu = fits.open("/Volumes/astroplates2/schmidt_telescope_lc/variable_search/gsvs_catalogue/20240905_gsvs_catalogue.fits")

In [22]:
pd.DataFrame(hdu[1].data).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58202 entries, 0 to 58201
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NAME                    58202 non-null  object 
 1   RA                      58202 non-null  float64
 2   DEC                     58202 non-null  float64
 3   SPECT_TYPE              58202 non-null  object 
 4   MIN_MAG_LIMIT           58202 non-null  object 
 5   MIN_MAG                 58202 non-null  float64
 6   MAX_MAG_LIMIT           58202 non-null  object 
 7   MAX_MAG                 58202 non-null  float64
 8   VARIABILITY_TYPE        58202 non-null  object 
 9   PERIOD_LIMIT            58202 non-null  object 
 10  PERIOD                  58202 non-null  float64
 11  PERIOD_FLAG             58202 non-null  object 
 12  VARIABLE_STAR_NUMBER    58202 non-null  int32  
 13  VARIABLE_STAR_CPT       58202 non-null  object 
 14  NOTE_FLAG               58202 non-null

In [16]:
pd.read_csv("/Volumes/astroplates2/schmidt_telescope_lc/variable_search/output/parsed_from_frames/alf-Aql_25-26.07.1988_10m_47S-2-1_objects.csv")

,id,x,y,ra,dec,flux,flux_err,mag,mag_err,flag,a,b
0,0,291.714955,1.147484,298.333088,17.556717,24301.383437,2753.160951,-10.964077,0.123006,16,3.134369,0.843677
1,1,955.759757,1.085002,297.250265,17.573785,20579.496563,2731.468197,-10.783587,0.144107,16,1.223082,0.849620
2,2,1243.081584,1.213598,296.781121,17.579104,21196.886250,2771.144069,-10.815680,0.141942,16,1.267716,0.864745
3,3,1329.267849,1.027848,296.640340,17.580836,29944.842656,2719.089427,-11.190805,0.098588,16,1.461335,0.920101
4,4,1485.622710,1.086028,296.384875,17.583105,62737.246250,2739.174376,-11.993814,0.047404,16,2.058546,0.965340
...,...,...,...,...,...,...,...,...,...,...,...,...
66882,13586,4245.296509,3810.802028,291.935111,11.606622,152354.268750,3021.493709,-12.957137,0.021532,0,3.113695,2.827332
66883,13587,4346.597673,3815.558982,291.774561,11.597827,29886.789063,2823.059221,-11.188698,0.102557,16,1.543064,1.063053
66884,13588,4536.412283,3815.375197,291.473691,11.595300,17844.298438,2872.657476,-10.628749,0.174787,16,1.454341,0.716828
66885,13589,4739.398728,3815.948664,291.152152,11.591055,25531.759688,2715.921232,-11.017702,0.115494,16,0.966516,0.846687


# Functions

## Catalogue

In [51]:
# Функция для фильтрации гигантского каталога по координатам
def filter_catalog_by_coordinates(frame_df, catalog_df):
    """
    Ограничивает гигантский каталог по минимальным и максимальным значениям координат из таблицы объектов с кадра.
    
    Parameters:
    - frame_df: DataFrame с объектами с кадра (должен содержать столбцы 'ra' и 'dec').
    - catalog_df: Гигантский каталог в виде DataFrame (содержит 'ra' и 'dec').
    
    Returns:
    - Ограниченный каталог по координатам.
    """
    # Получение минимальных и максимальных значений координат из таблицы объектов с кадра
    ra_min, ra_max = frame_df['ra'].min(), frame_df['ra'].max()
    dec_min, dec_max = frame_df['dec'].min(), frame_df['dec'].max()

    # Ограничение гигантского каталога по этим значениям
    filtered_catalog = catalog_df[(catalog_df['ra'] >= ra_min) & (catalog_df['ra'] <= ra_max) &
                                  (catalog_df['dec'] >= dec_min) & (catalog_df['dec'] <= dec_max)]

    return filtered_catalog

#Функция для сопоставления и обработки данных
def match_objects_and_catalog(frame_df, catalog_df, output_path, unmatched_output_path, radius):
    """
    Сопоставляет объекты с кадра с каталогом и создает две таблицы:
    1. Таблица с сопоставленными объектами и добавленными колонками из каталога.
    2. Таблица с объектами, которые не нашли себе пары.
    Также добавляет столбец 'match_status', где объекты с расстоянием > radius отмечены как "uncertain".

    Parameters:
    - frame_df: DataFrame с объектами с кадра (должен содержать столбцы 'ra' и 'dec').
    - catalog_df: Ограниченный каталог (после фильтрации по координатам).
    - output_path: Путь для сохранения результирующей таблицы с сопоставленными объектами.
    - unmatched_output_path: Путь для сохранения таблицы с объектами без совпадений.
    - radius: Угловое расстояние (в угловых секундах), выше которого объект считается сомнительным.
    
    Returns:
    - Сохраняет таблицы с совпавшими и несовпавшими объектами.
    """
    # Создаем SkyCoord для объектов с кадра и каталога
    frame_coords = SkyCoord(ra=frame_df['ra'].values * u.deg, dec=frame_df['dec'].values * u.deg)
    catalog_coords = SkyCoord(ra=catalog_df['ra'].values * u.deg, dec=catalog_df['dec'].values * u.deg)

    # Выполняем кросс-матч
    idx, d2d, d3d = match_coordinates_sky(frame_coords, catalog_coords)

    # Переименовываем колонки из первой таблицы с добавлением суффикса '_frame'
    frame_df = frame_df.add_suffix('_frame')

    # Создаем таблицу с совпавшими объектами
    matched_frame = frame_df.copy()

    # Добавляем колонки из каталога с суффиксом _cat
    for col in catalog_df.columns:
        matched_frame[col + '_cat'] = catalog_df.iloc[idx][col].values

    # Добавляем столбец с разницей в координатах (в угловых секундах)
    matched_frame['coord_diff_arcsec'] = d2d.arcsec

    # Добавляем столбец 'match_status', где отмечаем объекты как "uncertain", если расстояние больше радиуса
    matched_frame['match_status'] = ['uncertain' if diff > radius else 'matched' for diff in d2d.arcsec]

    # Сохраняем таблицу с совпавшими объектами
    matched_frame.to_csv(output_path, index=False)

    # Находим объекты без совпадений
    unmatched_mask = d2d.arcsec > radius  # Условие для неподходящих объектов
    unmatched_frame = frame_df[unmatched_mask]

    # Сохраняем таблицу с объектами без совпадений
    unmatched_frame.to_csv(unmatched_output_path, index=False)

    return matched_frame, unmatched_frame

def create_fits_with_metadata(matched_frame, catalog_fits_path, output_fits_path):
    """
    Создает FITS файл с метаданными для результирующей таблицы с данными из первого и второго каталогов.
    
    Parameters:
    - matched_frame: DataFrame с совпавшими объектами (содержит объединенные данные).
    - catalog_fits_path: Путь к исходному FITS файлу гигантского каталога (для извлечения метаданных).
    - output_fits_path: Путь для сохранения результирующей таблицы в FITS формате.
    """
    
    # Конвертируем таблицу pandas в astropy Table для работы с FITS
    table = Table.from_pandas(matched_frame)
    
    # Открываем FITS файл каталога для извлечения заголовков
    with fits.open(catalog_fits_path) as hdul:
        catalog_header = hdul[1].header  # Предполагается, что данные хранятся в HDU1

    # Добавление метаданных для колонок из первой таблицы (объекты с кадра)
    metadata_frame = {
        'id_frame': {'description': 'Object identifier', 'unit': ''},
        'x_frame': {'description': 'X coordinate on image', 'unit': 'pixels'},
        'y_frame': {'description': 'Y coordinate on image', 'unit': 'pixels'},
        'ra_frame': {'description': 'Right ascension of object', 'unit': 'degrees'},
        'dec_frame': {'description': 'Declination of object', 'unit': 'degrees'},
        'flux_frame': {'description': 'Flux of object', 'unit': 'arbitrary units'},
        'flux_err_frame': {'description': 'Flux error', 'unit': 'arbitrary units'},
        'mag_frame': {'description': 'Magnitude of object', 'unit': 'mag'},
        'mag_err_frame': {'description': 'Magnitude error', 'unit': 'mag'},
        'flag_frame': {'description': 'Quality flags', 'unit': ''},
        'a_frame': {'description': 'Semi-major axis', 'unit': 'pixels'},
        'b_frame': {'description': 'Semi-minor axis', 'unit': 'pixels'},
        'coord_diff_arcsec': {'description': 'Coordinate difference between objects', 'unit': 'arcsec'}
    }

    # Применяем метаданные к первой части таблицы (данные с кадра)
    for col in metadata_frame:
        if col in table.colnames:
            table[col].description = metadata_frame[col]['description']
            table[col].unit = metadata_frame[col]['unit']
    
    # Добавляем метаданные для колонок из второго каталога, используя заголовки FITS из оригинального каталога
    for col in catalog_header:
        cat_col_name = col + '_cat'
        if cat_col_name in table.colnames:
            table[cat_col_name].description = catalog_header.comments[col]  # Используем комментарии из FITS как описание
            if catalog_header[col] == 'TUNIT':  # Проверяем наличие единиц измерения
                table[cat_col_name].unit = catalog_header[col]
            else:
                table[cat_col_name].unit = ''

    # Сохраняем таблицу в формате FITS с метаданными
    table.write(output_fits_path, format='fits', overwrite=True)

## Process directory

In [8]:
def process_directory_export_objects(directory, output_directory, scale='zscale'):
    """
    Обрабатывает все FITS файлы в указанном каталоге, выполняя вычитание фона и извлечение объектов.
    """

        print(f"Processed {filename} ({i+1}/{total_files}) in {end_time - start_time:.2f} seconds.")
        print("-------------------------------------------")

## Plot parametres

In [10]:
plt.rcParams.update({
    'figure.figsize': (8, 6),  # Размеры графиков
    'figure.dpi': 100,  # Разрешение графиков
    'savefig.dpi': 300,  # Разрешение сохраненных изображений
    'font.size': 12,  # Размер шрифта
    'font.family': 'serif',  # Семейство шрифтов
    'axes.labelsize': 14,  # Размер шрифта для подписей осей
    'axes.titlesize': 14,  # Размер шрифта для заголовков
    'xtick.labelsize': 12,  # Размер шрифта для подписей меток на оси X
    'ytick.labelsize': 12,  # Размер шрифта для подписей меток на оси Y
    'legend.fontsize': 10,  # Размер шрифта для легенды
    'lines.linewidth': 1,  # Толщина линий
    'lines.markersize': 6,  # Размер маркеров
    'axes.grid': False,  # Сетка на графиках
    'grid.alpha': 0.25,  # Прозрачность сетки
    'grid.linestyle': '--',  # Стиль линии сетки
    'grid.color': 'gray',  # Цвет сетки
    'axes.axisbelow': True,  # Сетка ниже данных
    'image.cmap': 'viridis',  # Цветовая карта для изображений
    'errorbar.capsize': 3,  # Размер "колпачков" на графиках ошибок
    "legend.loc": 'best', 
})

# Implementation

In [ ]:
radius = 21.751767 #degrees taken from calculation "mean" radius for frames

In [12]:
# Укажи путь к директории с FITS файлами и директории для сохранения результатов
fits_directory = os.getcwd() + '/data/'
output_directory = os.getcwd() + '/output/'

In [ ]:
process_directory_export_objects(fits_directory, output_directory,scale='zscale')

In [10]:
hdu = fits.open("/Users/ildana/astro/fai/variable_search/catalogues/gaiadr3.gaia_source_more12.8less12.9.fits")

In [37]:
df["phot_variable_flag"].unique()

array(['NOT_AVAILABLE', 'VARIABLE'], dtype=object)

In [24]:
hdu[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional table                            
NAXIS1  =                 1380 / width of table in bytes                        
NAXIS2  =               555707 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group                                 
TFIELDS =                  153 / number of columns                              
EXTNAME = 'gaia_user_anonymous.2024-10-04-07-12-06-889332' / table name         
TTYPE1  = 'solution_id'        / label for column 1                             
TFORM1  = 'K       '           / format for column 1                            
TUCD1   = 'meta.version'       / VO Unified Content Descriptor for column 1     
TTYPE2  = 'designation'     